# Первичная настройка путей

In [1]:
from pathlib import Path


raw_data_path = Path("../data/raw")
preprocessed_file = Path("../data/preprocessed/image_only.csv")
images_dir = Path('../data/images')

In [9]:
import pandas as pd
if not preprocessed_file.exists():
    main_df = pd.DataFrame(columns=["image_name", "blocked"])
else: 
    main_df = pd.read_csv(preprocessed_file)

# Загрузка данных

## hearmeneigh/e621-rising-v3-small

In [ ]:
from vk_mod.data.download import load_from_huggingface
load_from_huggingface("hearmeneigh/e621-rising-v3-small")
temp_df = pd.read_csv("../data/raw/e621-rising-v3-small.csv")
temp_df.head(5)

c:\Users\ShapeCat\source\repos\ToxicPostDetection\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Unnamed: 0,source_id,source,image,tags,url,text,selector
0,0,3724100,e621,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"['genitals', 'animal_penis', 'humanoid_pointy_...",https://static1.e621.net/data/9b/9b/9b9b3a2a14...,genitals animal_penis humanoid_pointy_ears ani...,tier-1
1,1,2323551,e621,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"['crop_top', 'blush', 'clothed', 'solo', 'wolf...",https://static1.e621.net/data/b6/73/b673f9bd71...,crop_top blush clothed solo wolf_humanoid blac...,tier-2
2,2,3858728,e621,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"['genitals', 'traditional_media_artwork', 'clo...",https://static1.e621.net/data/4c/a6/4ca60e5ac9...,genitals traditional_media_artwork cloven_hoov...,tier-3
3,3,3972702,e621,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"['thigh_highs', 'score_below_500', 'purple_fur...",https://static1.e621.net/data/93/98/9398b7dfb4...,thigh_highs score_below_500 purple_fur chest_t...,tier-2
4,4,2535548,e621,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"['handsfree', 'antlers', 'score_above_500', 's...",https://static1.e621.net/data/9e/14/9e1452967b...,handsfree antlers score_above_500 score_above_...,tier-1


In [ ]:
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm

Image.MAX_IMAGE_PIXELS = None

images_dir.mkdir(exist_ok=True)
processed_data = []

for index, row in tqdm(temp_df.iterrows(), total=len(temp_df)):
    is_explicit = 1 if 'rating_explicit' in row['tags'] else 0
    try:
        response = requests.get(row['url'] , timeout=3)
        image = Image.open(BytesIO(response.content))
        if image.mode != 'RGB':
            image = image.convert('RGB')

        filename = f"e621v3{index}.jpg"     
        image.save(Path(images_dir, filename))        
        processed_data.append({
            'image_name': filename,
            'blocked': is_explicit
        })
    except Exception as e:
        #print(f"Ошибка обработки изображения {row['url']}: {e}")
        pass

temp_df = pd.DataFrame(processed_data)
display(temp_df.sample(5))
display(df.loc[:, df.columns == "blocked"].value_counts().reset_index(name="count"))

100%|██████████| 3747/3747 [1:59:00<00:00,  1.91s/it]  


,image_name,blocked
1445,e621v31521.jpg,1
3468,e621v33659.jpg,1
2353,e621v32494.jpg,1
1592,e621v31680.jpg,0
1504,e621v31584.jpg,1


,blocked,count


In [6]:
main_df = pd.concat([main_df, temp_df])
display(main_df)

,image_name,blocked
0,e621v30.jpg,1
1,e621v31.jpg,1
2,e621v32.jpg,1
3,e621v33.jpg,0
4,e621v34.jpg,1
...,...,...
3548,e621v33742.jpg,1
3549,e621v33743.jpg,0
3550,e621v33744.jpg,1
3551,e621v33745.jpg,0


# Очистка данных

In [12]:
main_df = (main_df
           .drop("Unnamed: 0", axis=1, errors="ignore")
           .drop_duplicates()
           .dropna(subset=["image_name", "blocked"]))
main_df['blocked'] = main_df['blocked'].astype(int)

In [13]:
display(main_df.loc[:, main_df.columns == "blocked"].value_counts().reset_index(name="count"))
print(main_df.shape)

,blocked,count
0,1,2859
1,0,694


(3553, 2)


# Сохранение

In [14]:
preprocessed_file.parent.mkdir(exist_ok=True)
main_df.to_csv(preprocessed_file, index=False)